In [3]:
# ===== GLOBAL CONFIG =====
import pandas as pd

YEAR = 2025

# Input files
GAME_INFO_FILE = f"game_info_{YEAR}.csv"
BOX_PATH = f"game_boxscores_{YEAR}.csv"

# Split outputs (based on dates)
PRE_INFO_FILE  = f"pre_info_{YEAR}.csv"
POST_INFO_FILE = f"post_info_{YEAR}.csv"

# Team stats
TEAM_STATS_FILE         = f"team_stats_{YEAR}.csv"
TEAM_STATS_UPDATED = f"team_stats_{YEAR}_updated.csv"

# Post-split with calculated diffs
POST_DIFF_FILE = f"post_info_{YEAR}_diff.csv"

In [4]:
"""
Split game_info_<YEAR>.csv into:
- pre_<YEAR>.csv  : games from Dec 1 (previous year) to Jan 9
- post_<YEAR>.csv : games from Jan 10 to Mar 10 (inclusive)
"""
PRE_FILE  = f"pre_{YEAR}.csv"
POST_FILE = f"post_{YEAR}.csv"

# Define date boundaries
PRE_START  = f"{int(YEAR)-1}-02-07"  # December 1 of previous year
PRE_END    = f"{YEAR}-02-06"
POST_START = f"2022-11-07"
POST_END   = f"{YEAR}-03-06"

# Load data
df = pd.read_csv(GAME_INFO_FILE)
df["game_day"] = pd.to_datetime(df["game_day"], errors="coerce")

# Apply filters
pre_df  = df[(df["game_day"] >= PRE_START) & (df["game_day"] <= PRE_END)]
post_df = df[(df["game_day"] >= POST_START) & (df["game_day"] <= POST_END)]

# Save files
pre_df.to_csv(PRE_FILE, index=False)
post_df.to_csv(POST_FILE, index=False)

print(f"✅ {YEAR} split → {len(pre_df)} pre | {len(post_df)} post")

✅ 2025 split → 1192 pre | 1489 post


In [5]:
"""
Create team_stats_<YEAR>.csv with a single column 'team'
containing teams that appear in both pre and post windows.
"""

import pandas as pd

pre  = pd.read_csv(PRE_FILE)
post = pd.read_csv(POST_FILE)

teams_pre  = set(pre["home_team"]).union(pre["away_team"])
teams_post = set(post["home_team"]).union(post["away_team"])

common_teams = sorted(list(teams_pre & teams_post))
pd.DataFrame({"team": common_teams}).to_csv(TEAM_STATS_FILE, index=False)

print(f"✅ {len(common_teams)} teams → {TEAM_STATS_FILE}")

✅ 339 teams → team_stats_2025.csv


In [6]:
"""
Augment team_stats_<YEAR>.csv with columns:
- home_game_id: list of pre-season home game_ids
- away_game_id: list of pre-season away game_ids
"""

import pandas as pd

teams = pd.read_csv(TEAM_STATS_FILE)
pre   = pd.read_csv(PRE_FILE)

home_games = pre.groupby("home_team")["game_id"].apply(list).to_dict()
away_games = pre.groupby("away_team")["game_id"].apply(list).to_dict()

teams["home_game_id"] = teams["team"].map(home_games).apply(lambda x: x if isinstance(x, list) else [])
teams["away_game_id"] = teams["team"].map(away_games).apply(lambda x: x if isinstance(x, list) else [])

teams.to_csv(TEAM_STATS_FILE, index=False)
print(f"✅ Added home/away game_id lists → {TEAM_STATS_FILE}")

✅ Added home/away game_id lists → team_stats_2025.csv


In [7]:
"""
Build nested dictionary for valid games:
games_dict[game_id][team_name] = {
    "team_total": <one-row DataFrame of numeric totals>,
    "player_stats": <player-level DataFrame for that team in that game>
}
"""

import pandas as pd, ast

def _parse_list(x):
    if isinstance(x, list): return x
    if pd.isna(x): return []
    s = str(x).strip()
    if s in ("", "[]"): return []
    return ast.literal_eval(s)

# Load boxscores
box = pd.read_csv(BOX_PATH)
box["game_id"] = pd.to_numeric(box["game_id"], errors="coerce").astype("Int64")
box = box.dropna(subset=["game_id"]).copy()
box["game_id"] = box["game_id"].astype(int)
box["team"]    = box["team"].astype(str)

# Load teams and collect valid game_ids
teams = pd.read_csv(TEAM_STATS_FILE)
teams["home_game_id"] = teams["home_game_id"].apply(_parse_list).apply(lambda L: [int(i) for i in L])
teams["away_game_id"] = teams["away_game_id"].apply(_parse_list).apply(lambda L: [int(i) for i in L])

valid_ids = set([gid for L in teams["home_game_id"] for gid in L] +
                [gid for L in teams["away_game_id"] for gid in L])
print(f"🎯 Valid pre-game IDs in team_stats: {len(valid_ids)}")

# Filter box
box = box[box["game_id"].isin(valid_ids)].copy()

# Build dict
games_dict = {}
skipped = 0

for gid, gdf in box.groupby("game_id"):
    tm = gdf["team"].dropna().unique()
    if len(tm) != 2:
        skipped += 1
        continue
    games_dict[gid] = {}
    numeric_cols = gdf.select_dtypes(include="number").columns
    for t in tm:
        df_team = gdf[gdf["team"] == t].copy()
        team_total = df_team[numeric_cols].sum(numeric_only=True).to_frame().T.reset_index(drop=True)
        games_dict[gid][t] = {"team_total": team_total, "player_stats": df_team.reset_index(drop=True)}

print(f"✅ games_dict built for {len(games_dict)} games | skipped: {skipped}")

🎯 Valid pre-game IDs in team_stats: 1192
✅ games_dict built for 1192 games | skipped: 0


In [8]:
"""
Remove any game_ids in team_stats that are not present in games_dict.
Overwrites team_stats_<YEAR>.csv with cleaned lists.
"""

import pandas as pd, ast

teams = pd.read_csv(TEAM_STATS_FILE)
teams["home_game_id"] = teams["home_game_id"].apply(_parse_list)
teams["away_game_id"] = teams["away_game_id"].apply(_parse_list)

games_dict_ids = set(map(int, games_dict.keys()))

def _keep_known(L): return [int(g) for g in L if int(g) in games_dict_ids]

before = teams["home_game_id"].apply(len).sum() + teams["away_game_id"].apply(len).sum()
teams["home_game_id"] = teams["home_game_id"].apply(_keep_known)
teams["away_game_id"] = teams["away_game_id"].apply(_keep_known)
after  = teams["home_game_id"].apply(len).sum() + teams["away_game_id"].apply(len).sum()

teams.to_csv(TEAM_STATS_FILE, index=False)
print(f"🧹 Cleaned team_stats: removed {before - after} invalid game_id refs.")

🧹 Cleaned team_stats: removed 0 invalid game_id refs.


In [9]:
"""
For each (game, team) block in games_dict, compute:
- poss  = FGA - OREB + TO + 0.475*FTA
- off_eff = 100 * PTS / poss
- def_eff = opponent's points per opponent poss
Also store basic shooting metrics:
- eFG% = (FGM + 0.5*3PM) / FGA
- TS%  = PTS / [2 * (FGA + 0.44*FTA)]
- 3PAr = 3PA / FGA
- FTr  = FTA / FGA
"""

import numpy as np
FT_FACTOR = 0.475

def _ensure(team_block, col):
    tt = team_block["team_total"]
    if col not in tt.columns or pd.isna(tt[col].iloc[0]):
        s = pd.to_numeric(team_block["player_stats"].get(col, 0), errors="coerce").fillna(0).sum()
        if tt.empty:
            team_block["team_total"] = pd.DataFrame([{col: float(s)}])
        else:
            team_block["team_total"].loc[:, col] = float(s)

def _compute_shooting(tt):
    # guard zeros
    FGA = float(tt.get("fga", [0])[0] if "fga" in tt else 0)
    FGM = float(tt.get("fgm", [0])[0] if "fgm" in tt else 0)
    P3M = float(tt.get("3pm", [0])[0] if "3pm" in tt else 0)
    P3A = float(tt.get("3pa", [0])[0] if "3pa" in tt else 0)
    FTA = float(tt.get("fta", [0])[0] if "fta" in tt else 0)
    efg = (FGM + 0.5*P3M) / FGA if FGA > 0 else np.nan
    ts  = (tt["pts"].iloc[0]) / (2 * (FGA + 0.44*FTA)) if (FGA + 0.44*FTA) > 0 else np.nan
    threepar = P3A / FGA if FGA > 0 else np.nan
    ftr  = FTA / FGA if FGA > 0 else np.nan
    return efg, ts, threepar, ftr

rows = []
for gid, teams_block in games_dict.items():
    if len(teams_block) != 2: continue
    t1, t2 = list(teams_block.keys())

    for t in (t1, t2):
        for c in ["fga","oreb","to","fta","pts","fgm","3pm","3pa"]:
            _ensure(teams_block[t], c)

    tt1, tt2 = teams_block[t1]["team_total"], teams_block[t2]["team_total"]

    for t_cur, t_opp in [(t1, t2), (t2, t1)]:
        cur = teams_block[t_cur]["team_total"].copy()
        opp = teams_block[t_opp]["team_total"].copy()

        fga, oreb, tov, fta = [float(pd.to_numeric(cur[c], errors="coerce").iloc[0]) for c in ["fga","oreb","to","fta"]]
        poss = fga - oreb + tov + FT_FACTOR * fta
        pts  = float(pd.to_numeric(cur["pts"], errors="coerce").iloc[0])

        poss_opp = float(pd.to_numeric(opp["fga"], errors="coerce").iloc[0] -
                         pd.to_numeric(opp["oreb"], errors="coerce").iloc[0] +
                         pd.to_numeric(opp["to"], errors="coerce").iloc[0] +
                         FT_FACTOR * pd.to_numeric(opp["fta"], errors="coerce").iloc[0])

        off_eff = 100.0 * pts / poss if poss > 0 else np.nan
        def_eff = 100.0 * float(pd.to_numeric(opp["pts"], errors="coerce").iloc[0]) / poss_opp if poss_opp > 0 else np.nan

        efg, ts, threepar, ftr = _compute_shooting(cur)

        games_dict[gid][t_cur]["team_total"].loc[:, "poss"]    = poss
        games_dict[gid][t_cur]["team_total"].loc[:, "off_eff"] = off_eff
        games_dict[gid][t_cur]["team_total"].loc[:, "def_eff"] = def_eff
        games_dict[gid][t_cur]["team_total"].loc[:, "efg_pct"] = efg
        games_dict[gid][t_cur]["team_total"].loc[:, "ts_pct"]  = ts
        games_dict[gid][t_cur]["team_total"].loc[:, "threepar"]= threepar
        games_dict[gid][t_cur]["team_total"].loc[:, "ftr"]     = ftr

        rows.append({"game_id": gid, "team": t_cur, "opp": t_opp,
                     "off_eff": off_eff, "def_eff": def_eff,
                     "efg_pct": efg, "ts_pct": ts, "threepar": threepar, "ftr": ftr})

eff_df = pd.DataFrame(rows)
print(f"✅ Computed per-game eff/shooting for {eff_df['game_id'].nunique()} games, {eff_df.shape[0]} team-rows")

✅ Computed per-game eff/shooting for 1192 games, 2384 team-rows


In [10]:
"""
Create per-team season averages and adjusted efficiencies,
then save to team_stats_<YEAR>_updated.csv
"""

import pandas as pd
import numpy as np

# Team means
team_means = eff_df.groupby("team")[["off_eff","def_eff","efg_pct","ts_pct","threepar","ftr"]].mean()

# League means for scaling
league_off = team_means["off_eff"].mean()
league_def = team_means["def_eff"].mean()

# Build opponent averages for adjustment
opp_means = eff_df.groupby("team")[["off_eff","def_eff"]].mean().rename(
    columns={"off_eff":"avg_off_eff","def_eff":"avg_def_eff"}
)

eff_adj = eff_df.merge(
    opp_means.rename(columns={"avg_off_eff":"avg_off_eff_opp", "avg_def_eff":"avg_def_eff_opp"}),
    left_on="opp", right_index=True, how="left"
)

eff_adj["adj_off_eff"] = (eff_adj["off_eff"] / eff_adj["avg_def_eff_opp"]) * league_off
eff_adj["adj_def_eff"] = (eff_adj["def_eff"] / eff_adj["avg_off_eff_opp"]) * league_def

# Season averages (adjusted + raw shooting)
team_adj_means = eff_adj.groupby("team")[["adj_off_eff","adj_def_eff"]].mean()
team_stats_means = team_means.join(team_adj_means, how="left")

# Merge into team_stats file
base = pd.read_csv(TEAM_STATS_FILE)
team_stats_updated = base.merge(
    team_stats_means.reset_index(),
    on="team", how="left"
)

team_stats_updated.to_csv(TEAM_STATS_UPDATED, index=False)
print(f"✅ Saved team season averages (raw & adjusted) → {TEAM_STATS_UPDATED}")

✅ Saved team season averages (raw & adjusted) → team_stats_2025_updated.csv


In [11]:
"""
Compute team luck from post_<YEAR>.csv and merge (luck_hometeam / luck_awayteam)
Then also compute net_rating for each side.
"""

import pandas as pd
import numpy as np

post = pd.read_csv(POST_FILE)

# point spread & booleans (ensure numeric)
post["point_spread"] = post["home_score"] - post["away_score"]

# Build per-team win/margin from post window
post["home_win"] = (post["point_spread"] > 0).astype(int)

home_df = post.copy()
home_df["team"] = home_df["home_team"]
home_df["margin"] = home_df["point_spread"]
home_df["win"] = home_df["home_win"]

away_df = post.copy()
away_df["team"] = away_df["away_team"]
away_df["margin"] = -away_df["point_spread"]
away_df["win"] = 1 - away_df["home_win"]

team_games = pd.concat([home_df, away_df], ignore_index=True)

summary = (team_games.groupby("team")
           .agg(actual_win_pct=("win","mean"),
                avg_margin=("margin","mean"))
           .reset_index())

summary["expected_win_pct"] = 1 / (1 + 10 ** (-summary["avg_margin"] / 10))
summary["luck"] = summary["actual_win_pct"] - summary["expected_win_pct"]

luck = summary[["team","luck"]]

# Merge season efficiencies from updated team file
stats = pd.read_csv(TEAM_STATS_UPDATED)

# Build home/away versions of season stats you want on POST_FILE
home_cols = {
    "team":"home_team",
    "off_eff":"off_eff_hometeam",
    "def_eff":"def_eff_hometeam",
    "adj_off_eff":"adj_off_eff_hometeam",
    "adj_def_eff":"adj_def_eff_hometeam",
    "efg_pct":"efg_pct_hometeam",
    "ts_pct":"ts_pct_hometeam",
    "threepar":"threepar_hometeam",
    "ftr":"ftr_hometeam"
}
away_cols = {k: v.replace("home","away") for k, v in home_cols.items()}

post_enriched = post.merge(stats.rename(columns=home_cols), on="home_team", how="left")
post_enriched = post_enriched.merge(stats.rename(columns=away_cols), on="away_team", how="left")

# Net ratings
post_enriched["net_rating_hometeam"] = post_enriched["adj_off_eff_hometeam"] - post_enriched["adj_def_eff_hometeam"]
post_enriched["net_rating_awayteam"] = post_enriched["adj_off_eff_awayteam"] - post_enriched["adj_def_eff_awayteam"]

# Luck (home/away)
post_enriched = post_enriched.merge(luck.rename(columns={"team":"home_team","luck":"luck_hometeam"}), on="home_team", how="left")
post_enriched = post_enriched.merge(luck.rename(columns={"team":"away_team","luck":"luck_awayteam"}), on="away_team", how="left")

post_enriched.to_csv(POST_FILE, index=False)
print(f"✅ Enriched {POST_FILE} with season stats, net ratings, and luck")

✅ Enriched post_2025.csv with season stats, net ratings, and luck


In [12]:
"""
Compute home_advantage_score per team from pre window and add to POST and POST_DIFF later.
home_advantage_score = avg_home_margin - avg_away_margin (excluding neutral)
"""

import pandas as pd

pre = pd.read_csv(PRE_FILE)
if "is_neutral" in pre.columns:
    pre = pre[pre["is_neutral"] == False]

pre["home_margin"] = pre["home_score"] - pre["away_score"]

home_avg = pre.groupby("home_team")["home_margin"].mean().reset_index().rename(
    columns={"home_team":"team","home_margin":"avg_home_margin"}
)
away_avg = pre.groupby("away_team")["home_margin"].mean().reset_index().rename(
    columns={"away_team":"team","home_margin":"avg_away_margin"}
)

adv = pd.merge(home_avg, away_avg, on="team", how="outer").fillna(0)
adv["home_advantage_score"] = adv["avg_home_margin"] - adv["avg_away_margin"]

# Merge only the home team's advantage into POST_FILE
post = pd.read_csv(POST_FILE)
post = post.merge(adv[["team","home_advantage_score"]].rename(columns={"team":"home_team","home_advantage_score":"home_advantage"}),
                  on="home_team", how="left")
post.to_csv(POST_FILE, index=False)
print("✅ Added 'home_advantage' to POST_FILE")

✅ Added 'home_advantage' to POST_FILE


In [13]:
"""
Compute home - away differences for key features and save post_<YEAR>_diff.csv.
Keeps core game info + diff columns + (optional) home_advantage.
"""

import pandas as pd

df = pd.read_csv(POST_FILE)

# Ensure booleans are numeric
for c in ["is_conference","is_neutral"]:
    if c in df.columns:
        df[c] = df[c].astype(int)

# Differences
pairs = [
    ("off_eff_hometeam", "off_eff_awayteam"),
    ("def_eff_hometeam", "def_eff_awayteam"),
    ("adj_off_eff_hometeam", "adj_off_eff_awayteam"),
    ("adj_def_eff_hometeam", "adj_def_eff_awayteam"),
    ("efg_pct_hometeam", "efg_pct_awayteam"),
    ("ts_pct_hometeam", "ts_pct_awayteam"),
    ("threepar_hometeam", "threepar_awayteam"),
    ("ftr_hometeam", "ftr_awayteam"),
    ("luck_hometeam", "luck_awayteam"),
    ("net_rating_hometeam","net_rating_awayteam"),
]

for a, b in pairs:
    if a in df.columns and b in df.columns:
        df[a.replace("_hometeam","_diff")] = df[a] - df[b]

# Select columns to keep
keep = [
    "home_team","away_team","home_score","away_score","point_spread",
    "is_conference","is_neutral","home_advantage"  # optional if computed
] + [c for c in df.columns if c.endswith("_diff")]

df = df[keep]
df.to_csv(POST_DIFF_FILE, index=False)
print(f"✅ Saved diff dataset → {POST_DIFF_FILE}")

✅ Saved diff dataset → post_info_2025_diff.csv


In [14]:
"""
Optional nonlinearity: add interaction terms to post_<YEAR>_diff.csv
- rating_luck_interaction = net_rating_diff * luck_diff
- shooting_strength_interaction = net_rating_diff * efg_pct_diff
"""

import pandas as pd

df = pd.read_csv(POST_DIFF_FILE)
for req in ["net_rating_diff","luck_diff","efg_pct_diff"]:
    if req not in df.columns:
        raise ValueError(f"Missing {req} in {POST_DIFF_FILE}")

df["rating_luck_interaction"] = df["net_rating_diff"] * df["luck_diff"]
df["shooting_strength_interaction"] = df["net_rating_diff"] * df["efg_pct_diff"]
df.to_csv(POST_DIFF_FILE, index=False)
print(f"✅ Added interaction terms → {POST_DIFF_FILE}")

✅ Added interaction terms → post_info_2025_diff.csv


In [15]:
# === 1) SETUP & SPLIT ================================================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load diff dataset from your global POST_DIFF_FILE
df = pd.read_csv(POST_DIFF_FILE)

# Choose features & target
features = [
    "net_rating_diff",
    "efg_pct_diff",
    "luck_diff",
    "adj_off_eff_diff",
    "home_advantage",               
    "rating_luck_interaction",      
    "shooting_strength_interaction",
    "is_conference",
    "is_neutral",
]
# Keep only columns that actually exist
features = [f for f in features if f in df.columns]

TARGET = "point_spread"
df = df.dropna(subset=features + [TARGET])

# Split features/target
X = df[features]
y = df[TARGET]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

# Scale (fit on train, transform both)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print(f"✅ Using {len(features)} features: {features}")
print(f"✅ Train size: {X_train.shape[0]} | Test size: {X_test.shape[0]}")

✅ Using 9 features: ['net_rating_diff', 'efg_pct_diff', 'luck_diff', 'adj_off_eff_diff', 'home_advantage', 'rating_luck_interaction', 'shooting_strength_interaction', 'is_conference', 'is_neutral']
✅ Train size: 1031 | Test size: 442


In [16]:
# === 2) MODEL TRAINING ===============================================
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.001, max_iter=10000),
    "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=10000),
    "HuberRegressor": HuberRegressor(epsilon=1.35, max_iter=1000),
    "GradientBoosting": GradientBoostingRegressor(
        n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
    ),
    "RandomForest": RandomForestRegressor(n_estimators=300, random_state=42),
}

results, trained = [], {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    trained[name] = model
    y_pred = model.predict(X_test_scaled)
    results.append({
        "Model": name,
        "MAE": mean_absolute_error(y_test, y_pred),
        "R²": r2_score(y_test, y_pred),
    })

res = pd.DataFrame(results).sort_values("MAE").reset_index(drop=True)
print("\n🏀 Model Performance Comparison:")
print(res.to_string(index=False))

# Keep the best model name for analysis chunk
best_model_name = res.iloc[0]["Model"]
print(f"\n🌟 Best by MAE: {best_model_name}")


🏀 Model Performance Comparison:
           Model      MAE       R²
  HuberRegressor 8.273064 0.605235
           Ridge 8.295803 0.604820
      ElasticNet 8.297378 0.604729
LinearRegression 8.298796 0.604646
           Lasso 8.299090 0.604629
    RandomForest 8.970935 0.559730
GradientBoosting 8.978402 0.560075

🌟 Best by MAE: HuberRegressor


In [17]:
# === 3) ANALYSIS ======================================================
import numpy as np

print("\n📌 Feature correlation matrix (train set):")
corr = pd.DataFrame(X_train, columns=features).corr()
print(corr.round(3))

# Coefficients for linear-type models (on standardized features)
def show_linear_coefs(name):
    if name in trained and hasattr(trained[name], "coef_"):
        coefs = pd.Series(trained[name].coef_, index=features).sort_values(key=np.abs, ascending=False)
        print(f"\n🔎 Top coefficients ({name}) on standardized features:")
        print(coefs.to_string())
    else:
        print(f"\n({name}) has no linear coefficients to display.")

for lin in ["LinearRegression", "Ridge", "Lasso", "ElasticNet", "HuberRegressor"]:
    show_linear_coefs(lin)

# Tree-based importance (if available)
def show_importance(name):
    if name in trained and hasattr(trained[name], "feature_importances_"):
        imps = pd.Series(trained[name].feature_importances_, index=features).sort_values(ascending=False)
        print(f"\n🌲 Feature importances ({name}):")
        print(imps.to_string())
    else:
        print(f"\n({name}) has no tree feature_importances_ to display.")

for tree in ["RandomForest", "GradientBoosting"]:
    show_importance(tree)

# Optional: permutation importance for the best model (more robust)
try:
    from sklearn.inspection import permutation_importance
    best_model = trained[best_model_name]
    perm = permutation_importance(best_model, X_test_scaled, y_test, scoring="neg_mean_absolute_error",
                                  n_repeats=10, random_state=42)
    perm_imps = pd.Series(-perm.importances_mean, index=features).sort_values(ascending=False)
    print(f"\n🎯 Permutation importance (ΔMAE when shuffled) for {best_model_name}:")
    print(perm_imps.to_string())
except Exception as e:
    print(f"\n(perm importance skipped) {e}")


📌 Feature correlation matrix (train set):
                               net_rating_diff  efg_pct_diff  luck_diff  \
net_rating_diff                          1.000         0.759     -0.532   
efg_pct_diff                             0.759         1.000     -0.484   
luck_diff                               -0.532        -0.484      1.000   
adj_off_eff_diff                         0.877         0.880     -0.490   
home_advantage                           0.879         0.720     -0.474   
rating_luck_interaction                  0.553         0.477     -0.459   
shooting_strength_interaction           -0.724        -0.786      0.308   
is_conference                              NaN           NaN        NaN   
is_neutral                               0.157         0.141     -0.101   

                               adj_off_eff_diff  home_advantage  \
net_rating_diff                           0.877           0.879   
efg_pct_diff                              0.880           0.720   
luck_

In [18]:
# === 2.5) LEARNING CURVE (plug-and-play) ==============================
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.base import clone

def learning_curve_mae(model, Xtr, ytr, Xte, yte, sizes=None, repeats=8, random_state=42):
    """
    Trains `model` on increasing fractions of (Xtr, ytr) and evaluates MAE on (Xte, yte).
    Returns sizes, mean MAE per size, and std MAE per size.
    """
    if sizes is None:
        sizes = np.linspace(0.1, 1.0, 10)
    rng = np.random.default_rng(random_state)

    means, stds = [], []
    ytr_np = ytr.to_numpy() if hasattr(ytr, "to_numpy") else np.asarray(ytr)
    for s in sizes:
        n = max(10, int(len(Xtr) * s))
        maes = []
        for _ in range(repeats):
            idx = rng.choice(len(Xtr), n, replace=False)
            m = clone(model)
            m.fit(Xtr[idx], ytr_np[idx])
            pred = m.predict(Xte)
            maes.append(mean_absolute_error(yte, pred))
        means.append(float(np.mean(maes)))
        stds.append(float(np.std(maes)))
    return np.array(sizes), np.array(means), np.array(stds)

# Pick the best model from your leaderboard (already computed above)
best_model = trained[best_model_name]

# Run learning curves for the best model and a Ridge baseline
sizes, best_mean, best_std = learning_curve_mae(best_model, X_train_scaled, y_train, X_test_scaled, y_test)
_,     ridge_mean, ridge_std = learning_curve_mae(Ridge(alpha=1.0), X_train_scaled, y_train, X_test_scaled, y_test)

# Pretty print
print("\n📈 Learning Curve (MAE ± sd)")
print("Size   Best_MAE  ±sd     Ridge_MAE  ±sd")
for s, bm, bs, rm, rs in zip(sizes, best_mean, best_std, ridge_mean, ridge_std):
    print(f"{int(s*100):>3d}%   {bm:8.3f} ±{bs:4.3f}   {rm:10.3f} ±{rs:4.3f}")

# Heuristic verdict
def improvement(means, sizes_arr):
    # improvement from ~50% to 100%
    mid_idx = np.argmin(np.abs(sizes_arr - 0.5))
    return means[mid_idx] - means[-1]

best_improve = improvement(best_mean, sizes)
ridge_improve = improvement(ridge_mean, sizes)
tail_slope = best_mean[-2] - best_mean[-1] if len(best_mean) > 1 else 0.0

print("\n🧭 Diagnosis:")
print(f"- Best model improvement 50%→100%: {best_improve:.3f} MAE")
print(f"- Ridge improvement 50%→100%     : {ridge_improve:.3f} MAE")
print(f"- Last-step improvement (best)    : {tail_slope:.3f} MAE")

if best_improve >= 0.30 or tail_slope > 0.05:
    verdict = "📦 More data likely helps (curve still descending)."
elif best_improve < 0.10 and abs(tail_slope) < 0.02:
    verdict = "🧪 Curve is flat: focus on richer features (tempo/recent form/rest/travel/etc.)."
else:
    verdict = "⚖️ Mixed: modest data gains possible, but new features will likely matter more."

print(f"\n✅ Verdict: {verdict}")


📈 Learning Curve (MAE ± sd)
Size   Best_MAE  ±sd     Ridge_MAE  ±sd
 10%      8.658 ±0.207        8.665 ±0.212
 20%      8.417 ±0.123        8.417 ±0.125
 30%      8.257 ±0.070        8.286 ±0.066
 40%      8.369 ±0.123        8.370 ±0.104
 50%      8.318 ±0.062        8.334 ±0.094
 60%      8.294 ±0.036        8.320 ±0.039
 70%      8.315 ±0.018        8.338 ±0.024
 80%      8.289 ±0.030        8.303 ±0.025
 90%      8.279 ±0.028        8.299 ±0.029
100%      8.273 ±0.000        8.296 ±0.000

🧭 Diagnosis:
- Best model improvement 50%→100%: 0.045 MAE
- Ridge improvement 50%→100%     : 0.038 MAE
- Last-step improvement (best)    : 0.006 MAE

✅ Verdict: 🧪 Curve is flat: focus on richer features (tempo/recent form/rest/travel/etc.).
